In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm 

tqdm.pandas()

In [2]:
df = pd.read_csv("q_quora.csv")
df.sample(5)

C:\Users\HP\AppData\Local\Temp\ipykernel_23276\3667460292.py:1: DtypeWarning: Columns (7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("q_quora.csv")


,id,qid1,qid2,question1,question2,is_duplicate,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
91063,91063,180803,180804,"If I delete the messages on WhatsApp, is it al...",What are the signs in Whatsapp when they block...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191114,191114,377295,377296,What is Supply Chain Management?,What is a supply chain management?,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352396,352396,690286,690287,I feel my abdomen full of liquid and sloshing ...,I just ate a full pack of Maryland chocolate c...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58448,58448,116268,116269,Could someone be able to qualify for the US Ar...,How can I redeem/sell uti mep (master equity p...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150573,150573,297871,297872,"For an international student, which countries ...",Which countries offer free university educatio...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df.iloc[:, 3:6]

<h1>Start with text Pre-processing</h1>

In [4]:
df.sample(5)

,question1,question2,is_duplicate
389308,What is the equivalent Indian shoe size for th...,Is a size 8 1/2 in shoes pretty big for a woman?,0
273120,Where can I buy a ticket to Antarctica?,Where can I buy flight tickets for Antarctica?,1
392822,"If a parallel universe does exist, which one d...",If an infinite number of parallel universes do...,0
2074,"Why are ""bad boys"" so attractive?",Are girls attracted to boys as much as boys ar...,0
30611,What does it feel like to have histrionic pers...,What does histrionic personality disorder feel...,1


In [5]:
def make_int(num):
    try:
        return int(num)
    except:
        return -1
        
df['is_duplicate'] = df['is_duplicate'].apply(make_int)

In [6]:
df['is_duplicate'].value_counts()

is_duplicate
 0    254748
 1    149267
-1       336
Name: count, dtype: int64

In [7]:
df = df.drop(df[df['is_duplicate'] == -1].index)
df['is_duplicate'].value_counts()

is_duplicate
0    254748
1    149267
Name: count, dtype: int64

In [8]:
df.shape

(404015, 3)

In [9]:
df.sample(5)

,question1,question2,is_duplicate
96451,Are there any European universities that have ...,If you couldn't pass on any money to your kids...,0
282810,What is the best interview questions asked by ...,What are some good interview questions to ask ...,0
280939,What is the expected KVPY 2016 SA cut off?,What is the expected cut off for KVPY SA 2016?,1
199867,What do Pakistani people think about uri attac...,What do Pakistani residents think of Uri attack?,1
68601,Why do many people believe that humans are at ...,Why aren't humans at the top of the food chain?,0


In [10]:
df.drop(df[df['question1'].isna()].index, inplace = True)
df.drop(df[df['question2'].isna()].index, inplace = True)
df['question2'].isna().sum()

0

In [11]:
X = df.iloc[:, :2]
y = df.iloc[:, 2]
X.head(5)

,question1,question2
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


<h2>Lower Casing</h2>

In [12]:
X['question1'] = X['question1'].apply(lambda x : x.lower())
X['question2'] = X['question2'].apply(lambda x : x.lower())

df.head(5)

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


<h2>Remove HTML tags and URLs</h2>
<h4>Since there are no HTML tags and URLs we can move forward</h4>

<h2>Remove Punctuation marks</h2>

In [13]:
import string

punctuation_marks = string.punctuation

# We excluded '!' & '@' from punctuations as it could give useful information. 
punctuation_marks = punctuation_marks[1:20] + punctuation_marks[21:]

def remove_punctuations(text):
    return text.translate(str.maketrans(punctuation_marks, ' '*len(punctuation_marks)))

X['question1'] = X['question1'].apply(remove_punctuations)
X['question2'] = X['question2'].apply(remove_punctuations)
X.head(10)

,question1,question2
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...
2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,why am i mentally very lonely? how can i solve...,find the remainder when math 23 24 math i...
4,which one dissolve in water quikly sugar salt...,which fish would survive in salt water?
5,astrology i am a capricorn sun cap moon and c...,i m a triple capricorn sun moon and ascendan...
6,should i buy tiago?,what keeps childern active and far from phone ...
7,how can i be a good geologist?,what should i do to be a great geologist?
8,when do you use シ instead of し?,when do you use instead of and ?
9,motorola company can i hack my charter moto...,how do i hack motorola dcx3400 for free internet?


<h2>Chat Word Removal</h2>

<h4>Not much required as didn't find any</h4>

In [14]:
# import csv
# import re

# from tqdm import tqdm 

# tqdm.pandas()

# # Git hub link for the same can be found here : https://github.com/rishabhverma17/sms_slang_translator
# def chat_word_converter(user_string):
#     user_string = user_string.split(" ")
#     j = 0
#     for _str in user_string:
#         # File path which consists of Abbreviations.
#         fileName = "slang.txt"
#         # File Access mode [Read Mode]
#         accessMode = "r"
#         with open(fileName, accessMode) as myCSVfile:
#             # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
#             dataFromFile = csv.reader(myCSVfile, delimiter="=")
#             # Removing Special Characters.
#             _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
#             for row in dataFromFile:
#                 # Check if selected word matches short forms[LHS] in text file.
#                 if _str.upper() == row[0]:
#                     # If match found replace it with its Abbreviation in text file.
#                     user_string[j] = row[1]
#             myCSVfile.close()
#         j = j + 1
#     # Replacing commas with spaces for final output.
#     return ' '.join(user_string).lower()

# # print(chat_word_converter("Hi bro ltns"))

# X['question1'] = X['question1'].progress_apply(chat_word_converter)
# X['question2'] = X['question2'].progress_apply(chat_word_converter)
# X.sample(5)

<h2>Spelling checking</h2>

In [15]:
from spello.model import SpellCorrectionModel
import pickle
sp = SpellCorrectionModel(language = "en")
sp.load("en.pkl")

D:\Leaning_NLP\myenv\Lib\site-packages\spello\model.py:301: UserWarning: This model was saved on spell<1.3.0. As such due to a bug in previous versions, none of customisations made to the config at the time of training were saved along with the model. It is recommended to load the model, apply all required customizations to config and save it again. E.g.

from spello.model import SpellCorrectionModel 
sp = SpellCorrectionModel(language='en')  
sp.load('/home/ubuntu/model.pkl')
sp.config.min_length_for_spellcorrection = 4 # default is 3
sp.config.max_length_for_spellcorrection = 12 # default is 15
sp.save(model_save_dir='/home/ubuntu/')

After this the model will load without any warnings

  warnings.warn("This model was saved on spell<1.3.0. As such due to a bug in previous versions, "


In [ ]:
def spell_correction(text):
    p = sp.spell_correct(text)
    return p['spell_corrected_text']

X['question1'] = X['question1'].progress_apply(spell_correction)
X['question2'] = X['question2'].progress_apply(spell_correction)

 33%|████████████████████████                                                  | 131633/404012 [02:22<05:42, 794.73it/s]

In [ ]:
X.head(10)

<h2>Remove Stop Words</h2>

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
en = stopwords.words("english")

In [ ]:
# Except not we take all the stop words as not will play an in important role in determining sentiment of the questions

new_en = []
for word in en:
    if word not in  ["not", "isn't"] :
        new_en.append(word)

en = new_en

In [ ]:
en

In [ ]:
def remove_stopwords(text):
    new_text = []
    words = text.split(" ")
    for word in words:
        if word not in en:
            new_text.append(word)
    new_text = new_text[:]      
    return " ".join(new_text)     

In [ ]:
X['question1'] = X['question1'].progress_apply(remove_stopwords)
X['question2'] = X['question2'].progress_apply(remove_stopwords)

In [ ]:
X.head(10)

<h2>Handeling Emojis</h2>

<h4>There are no emojis so not needed</h4>

<h2>Tokenization</h2>


In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def tokenizer(text):
    tokens = nlp(text)
    rtn = []
    for token in tokens:
        rtn.append(str(token))
    return rtn

In [ ]:
X['question1'] = X['question1'].progress_apply(tokenizer)
X['question2'] = X['question2'].progress_apply(tokenizer)

In [ ]:
X.head(10)

<h4>I tried using spacy but it was taking a lot of time, first let's do it with split method. First '?' and '!' needs to be seperated. </h4>

In [ ]:
import re

def tokenizer(text):
    text = re.findall(r'[^?]+|[^!]+|\!|\?', text)
    lst = []
    for txt in text:
        txt = re.split(r'\s+', txt)
        for w in txt:
            lst.append(w)
    return lst

In [ ]:
X['question1'] = X['question1'].progress_apply(tokenizer)
X['question2'] = X['question2'].progress_apply(tokenizer)

In [ ]:
X.head(10)

<h2>Stemming</h2>

In [ ]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()

def stemmer(words):
    rtn = []
    for word in words:
        rtn.append(ps.stem(word))

    return rtn

In [ ]:
X['question1'] = X['question1'].progress_apply(stemmer)
X['question2'] = X['question2'].progress_apply(stemmer)

In [ ]:
X.head(10)

<h1>Now Since we have done Pre-Preocessing, Now word vectorisation</h1>

<h2>We can use BagOfWords</h2>

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
# import and instantiate the vectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

# vectorize the lemmatized text
X = cv.fit_transform(df.processed_text)
print(X)